In [2]:
import os
import pandas as pd
import numpy as np

def normalize_and_process_csvs(input_dir, output_dir):
    """
    Processes CSV files in a directory, normalizes their rows, selects the first 512 signal columns, 
    and saves the results to a new directory.

    Args:
      input_dir (str): The path to the directory containing the input CSV files.
      output_dir (str): The path to the directory where the processed CSV files will be saved.
    """

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for filename in os.listdir(input_dir):
        if filename.endswith(".csv"):
            input_path = os.path.join(input_dir, filename)
            output_path = os.path.join(output_dir, filename)

            try:
                df = pd.read_csv(input_path)

                # 1. Initial normalization of all rows
                df.iloc[:, 1:] = df.iloc[:, 1:].div(df.iloc[:, 1:].sum(axis=1), axis=0) # avoid division by zero if sum of row is 0

                # 2. Select the first 512 Signal Value columns + the Segment ID column 
                selected_columns = [df.columns[0]] + list(df.columns[1:1025])
                df = df[selected_columns]


                # 3. Normalize again with the first 512 columns
                #df.iloc[:, 1:] = df.iloc[:, 1:].div(df.iloc[:, 1:].sum(axis=1), axis=0) # avoid division by zero if sum of row is 0

                # 4. Save the modified DataFrame
                df.to_csv(output_path, index=False)
                print(f"Processed and saved: {output_path}")


            except Exception as e:
                print(f"Error processing {filename}: {e}")

if __name__ == "__main__":
    input_directory = '/Users/jothamwambi/Downloads/Strong-Tread Time-Domain Segments'  # Replace with the path to your CSV files
    output_directory = '/Users/jothamwambi/Projects/tire_pressure_analysis/Pulse_Width_Analysis/Data/Mounted/11R22.5 (702ZE-i)/10 wheels/Strong-Tread Time-Domain Segments' # the output csvs are saved in this directory

    normalize_and_process_csvs(input_directory, output_directory)

Processed and saved: /Users/jothamwambi/Projects/tire_pressure_analysis/Pulse_Width_Analysis/Data/Mounted/11R22.5 (702ZE-i)/10 wheels/Strong-Tread Time-Domain Segments/Strong Tread Time Domain segment 700-POS-9 .csv
Processed and saved: /Users/jothamwambi/Projects/tire_pressure_analysis/Pulse_Width_Analysis/Data/Mounted/11R22.5 (702ZE-i)/10 wheels/Strong-Tread Time-Domain Segments/Strong Tread Time Domain segment 600-POS-4 .csv
Processed and saved: /Users/jothamwambi/Projects/tire_pressure_analysis/Pulse_Width_Analysis/Data/Mounted/11R22.5 (702ZE-i)/10 wheels/Strong-Tread Time-Domain Segments/Strong Tread Time Domain segment 800-POS-1 .csv
Processed and saved: /Users/jothamwambi/Projects/tire_pressure_analysis/Pulse_Width_Analysis/Data/Mounted/11R22.5 (702ZE-i)/10 wheels/Strong-Tread Time-Domain Segments/Strong Tread Time Domain segment 700-POS-5 .csv
Processed and saved: /Users/jothamwambi/Projects/tire_pressure_analysis/Pulse_Width_Analysis/Data/Mounted/11R22.5 (702ZE-i)/10 wheels/St

## Extract File name and save to Excel

In [3]:
import os
import pandas as pd
import glob
import re

def extract_wav_data(directory):
    """
    Extracts filenames and tire pressure from .wav files in a directory.

    Args:
        directory (str): The path to the directory containing the .wav files.

    Returns:
        pandas.DataFrame: A DataFrame with 'Filename' and 'TirePressure' columns,
                         or None if no matching .wav files are found.
    """
    wav_files = glob.glob(os.path.join(directory, '*.wav'))
    data_list = []  # List to store dictionaries of file data

    if not wav_files:
        print(f"No .wav files found in the directory: {directory}")
        return None  # Return None if no files found

    for filepath in wav_files:
        filename = os.path.basename(filepath)  # Get filename
        name_without_extension = os.path.splitext(filename)[0]
        # Regex to match the pattern _(pressure)_ and extract the number
        match = re.search(r'_(\d+)_', filename)

        pressure = None  # Initialize pressure to None
        if match:
             extracted_pressure = int(match.group(1)) #Extract the match

             #Check if the extracted number corresponds to one of the tire pressures
             if extracted_pressure in (500, 600, 700, 800, 850, 900):
                pressure = extracted_pressure  # Assign the extracted pressure

        data_list.append({'Filename': name_without_extension, 'TirePressure': pressure}) #Append dictionary to the list


    df = pd.DataFrame(data_list)
    return df


def save_to_excel(df, output_path):
    """
    Saves a pandas DataFrame to an Excel file.

    Args:
        df (pandas.DataFrame): The DataFrame to save.
        output_path (str): The path where the Excel file should be saved.
    """
    if df is not None:
        df.to_excel(output_path, index=False, engine='openpyxl')
        print(f"Extracted data saved to: {output_path}")

if __name__ == "__main__":
     # Get Directory path from User
    directory_path = input("Enter the directory path containing the WAV files: ")
    output_excel_path = "/Users/jothamwambi/Projects/tire_pressure_analysis/Pulse_Width_Analysis/Data/Mounted/wav_data.xlsx"  # Output Excel file name

    extracted_data_df = extract_wav_data(directory_path)

    if extracted_data_df is not None:
        save_to_excel(extracted_data_df, output_excel_path)

Extracted data saved to: /Users/jothamwambi/Projects/tire_pressure_analysis/Pulse_Width_Analysis/Data/Mounted/wav_data.xlsx
